# 0. import libraries

In [8]:
import time
start_time = time.time()
from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd
np.random.seed(32)
os.environ['OMP_NUM_THREADS'] = '8'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPool1D, concatenate, SpatialDropout1D
from keras.layers import Layer, InputSpec
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K

In [9]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
  def __init__(self, validation_data=(), interval=1):
    super(Callback, self).__init__()

    self.interval = interval
    self.X_val, self.y_val = validation_data

  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.interval == 0:
      y_pred = self.model.predict(self.X_val, verbose=0)
      score = roc_auc_score(self.y_val, y_pred)
      print(f'\n ROC-AUC - epoch: {epoch+1} - score: {score:.6f}\n')

In [10]:
NLP_ENG_DIR = '/content/drive/MyDrive/NLP/ENG/'
WV_DIR = NLP_ENG_DIR + 'wordvector/'
BASE_DIR = NLP_ENG_DIR + 'Jigsaw1/'
DATA_DIR = BASE_DIR + 'input/'
OUTPUT_DIR = BASE_DIR + 'output/'
MODEL_DIR = BASE_DIR + 'model/'

In [13]:
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')

embed_size = 300 # size of vector for each word
max_feature = 100000 # number of words to use as feature
max_len = 150 # sequence len

In [14]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
train.comment_text.fillna('no comment')
test.comment_text.fillna('no comment')
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=.1)

In [15]:
raw_text_train = X_train.comment_text.str.lower()
raw_text_valid = X_valid.comment_text.str.lower()
raw_text_test = test.comment_text.str.lower()

tk = Tokenizer(num_words=max_feature, lower=True)
tk.fit_on_texts(raw_text_train)

X_train['comment_seq'] = tk.texts_to_sequences(raw_text_train)
X_valid['comment_seq'] = tk.texts_to_sequences(raw_text_valid)
test['comment_seq'] = tk.texts_to_sequences(raw_text_test)

X_train = pad_sequences(X_train.comment_seq, maxlen=max_len)
X_valid = pad_sequences(X_valid.comment_seq, maxlen=max_len)
test = pad_sequences(test.comment_seq, maxlen=max_len)

In [18]:
def get_coefs(word, *arr):
  return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(WV_DIR+'glove/glove.840B.300d.txt'))

In [19]:
word_index = tk.word_index
nb_words = min(max_feature, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_feature: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [27]:
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

file_path = MODEL_DIR + 'bigru_cnn_pooling.hdf5'
check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1,
                              save_best_only=True, mode='min')
ra_val = RocAucEvaluation(validation_data=(X_valid, y_valid), interval=1)
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=5)

def build_model(lr=0.0, lr_d=0.0, units=0, dr=0.0):
  inp = Input(shape=(max_len, ))
  x = Embedding(max_feature, embed_size, weights = [embedding_matrix], trainable=False)(inp)
  x = SpatialDropout1D(dr)(x)

  x = Bidirectional(GRU(units, return_sequences=True))(x)
  x = Conv1D(64, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x)
  
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  
  x = concatenate([avg_pool, max_pool])

  x = Dense(6, activation='sigmoid')(x)
  model = Model(inputs=inp, outputs=x)
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr, decay=lr_d), metrics=['accuracy'])
  history = model.fit(X_train, y_train, batch_size=128, epochs=4, validation_data=(X_valid, y_valid),
                      verbose=1, callbacks=[ra_val, check_point, early_stop])
  model = load_model(file_path)
  return model

In [28]:
model = build_model(lr = 1e-3, lr_d = 0, units = 128, dr = 0.2)
pred = model.predict(test, batch_size = 1024, verbose = 1)

Epoch 1/4


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 [==============================] - ETA: 0s - loss: 0.0544 - accuracy: 0.8607
 ROC-AUC - epoch: 1 - score: 0.988470


Epoch 1: val_loss improved from inf to 0.04576, saving model to /content/drive/MyDrive/NLP/ENG/Jigsaw1/model/bigru_cnn_pooling.hdf5
1122/1122 [==============================] - 53s 33ms/step - loss: 0.0544 - accuracy: 0.8607 - val_loss: 0.0458 - val_accuracy: 0.6755
Epoch 2/4
1121/1122 [============================>.] - ETA: 0s - loss: 0.0427 - accuracy: 0.9038
 ROC-AUC - epoch: 2 - score: 0.989988


Epoch 2: val_loss improved from 0.04576 to 0.04125, saving model to /content/drive/MyDrive/NLP/ENG/Jigsaw1/model/bigru_cnn_pooling.hdf5
1122/1122 [==============================] - 37s 33ms/step - loss: 0.0427 - accuracy: 0.9035 - val_loss: 0.0413 - val_accuracy: 0.6020
Epoch 3/4
1121/1122 [============================>.] - ETA: 0s - loss: 0.0402 - accuracy: 0.8992
 ROC-AUC - epoch: 3 - score: 0.990376


Epoch 3: val_loss did not improve from 0.04125
1122/1122 [===

In [29]:
submission = pd.read_csv(DATA_DIR + "sample_submission.csv")
submission[list_classes] = (pred)
submission.to_csv(OUTPUT_DIR+"bigru_cnn_pooling.csv", index = False)

In [30]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997194,0.514851,0.975953,0.128452,0.915158,0.604232
1,0000247867823ef7,0.000424,0.000030,0.000381,0.000006,0.000188,0.000054
2,00013b17ad220c46,0.000372,0.000123,0.000424,0.000012,0.000148,0.000142
3,00017563c3f7919a,0.000116,0.000005,0.000153,0.000049,0.000037,0.000010
4,00017695ad8997eb,0.008779,0.000275,0.002671,0.000109,0.001294,0.000157


In [32]:
# from google.colab import files
# files.upload()

# !mkdir -p ~/.kaggle//
# !cp kaggle.json ~/.kaggle/
# # Permission Warning이 발생하지 않도록 해줍니다.
# !chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f /content/drive/MyDrive/NLP/ENG/Jigsaw1/output/bigru_cnn_pooling.csv -m "bigru_pooling_cnn"

Saving kaggle.json to kaggle.json
100% 13.7M/13.7M [00:06<00:00, 2.21MB/s]
Successfully submitted to Toxic Comment Classification Challenge